# Geberation Results

The goal of this notebook is to compare the text generations when using the traditional RAG and when usign the re-ranking with datamodels


In [11]:
import torch
import json
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from dmcr.evaluators.Rouge_L_evaluator import Rouge_L_evaluator

## Load Data

In [28]:
GENERATIONS_PATH = "generations"
rag_generations = json.load(open(f"{GENERATIONS_PATH}/generations.json"))
datamodels_generations = json.load(open(f"{GENERATIONS_PATH}/datamodels_generations.json"))

In [3]:
questions = pl.read_ipc("questions.feather")

In [33]:
questions[5:15]

example_id,question,answers,text,idx_gold_in_corpus
i64,str,list[str],str,i64
2760865417584086994,"""do veins carry blood to the he…","[""to""]","""Veins are blood vessels that c…",20970740
-3805339726851223343,"""who played charlie bucket in t…","[""Peter Gardner Ostrum""]","""Peter Gardner Ostrum ( / ˈoʊst…",20970741
-245123639209390252,"""what is 1 radian in terms of p…","[""1 / 2π""]","""Conversion of common angles Tu…",20970742
2126085010748850659,"""when does season 5 of bates mo…","[""February 20 , 2017""]","""The fifth and final season of …",20970743
-7375519584806382465,"""how many episodes are in serie…","[""seven""]","""The seventh season of the fant…",20970744
-8791189758516265195,"""who is next in line to be the …","[""Charles , Prince of Wales""]","""Queen Elizabeth II is the sove…",20970745
3564169539206178376,"""who is in charge of enforcing …","[""United States Civil Service Commission""]","""The Pendleton Civil Service Re…",20970746
-3481854117164495809,"""what is the name of latest ver…","[""Oreo""]","""Code name Version number Initi…",20970747
-4144729966148354479,"""why was there so much interest…","[""sugar markets""]","""After the American Civil War a…",20970748


In [34]:
results = {
    "idx": [],
    "mean_metric_rag": [],
    "mean_metric_datamodels": [],
}
evaluator = Rouge_L_evaluator()


for i in range(len(rag_generations)):
    rag_i_results = []
    datamodels_i_results = []
    results["idx"].append(i)
    for j in range(len(rag_generations[str(i)])):

        max_rag = 0
        max_datamodels = 0
        
        for res in  questions[i]["answers"].to_numpy().flatten()[0]:

            metric_rag = evaluator.evaluate(np.array([res]), np.array([str(rag_generations[str(i)][j])]))
            metric_datamodels = evaluator.evaluate(np.array([res]), np.array([str(datamodels_generations[str(i)][j])]))
            
            max_rag = max(max_rag, metric_rag[0])
            max_datamodels = max(max_datamodels, metric_datamodels[0])

        rag_i_results.append(max_rag)
        datamodels_i_results.append(max_datamodels)

    results["mean_metric_rag"].append(np.mean(rag_i_results))
    results["mean_metric_datamodels"].append(np.mean(datamodels_i_results))
    df_results = pl.DataFrame(results)
    df_results.write_ipc("results.feather")

df_results = pl.DataFrame(results)

In [8]:
rag_generations[str(i)][j]

'According to the Death Penalty Information Center, there are'

In [9]:
np.array([res])

array(['2,718'], dtype='<U5')

In [39]:
results = pl.read_ipc("results.feather")
rag_metric = results["mean_metric_rag"].to_numpy().flatten()
datamodels_metric = results["mean_metric_datamodels"].to_numpy().flatten()

In [43]:
print(np.mean(rag_metric), np.mean(datamodels_metric))

0.05067113997113996 0.05640779220779221
